# Project 1 - Part2

# Data Cleaning

In [1]:
# import statements
import pandas as pd

In [2]:
# load data
file = r"D:\CodingDojo\PandasForDataManipulation\food-sales-predictions-kb\sales_predictions.csv"

df = pd.read_csv(file)

# dataframe preview
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


## Dataframe Info

### Dataframe Observations Notes
 * 8523 entries
 * two ID columns, no nulls, but not unique
 * nulls are in two columns (item_weight is numeric and outlet_size is categorical)
 * outlet_type column can possibly be expanded
 * duplicates exist in categorical columns
 * irregular spellings in item_fat_content

### Irregular Spelling
#### Item_Fat_Content Column  
 * normalize category spellings
 * use `.replace()`

In [4]:
# examine Item_Fat_Content column elements
df['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [5]:
# replace elements to normalize
df['Item_Fat_Content'].replace(['LF','low fat'],'Low Fat',inplace=True)
df['Item_Fat_Content'].replace('reg','Regular',inplace=True)

In [6]:
# verify replacements
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

### Missing Values
The approach to fill in missing data can be determined by exploring relationships within the data

In [7]:
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

#### Explore data elements to see relationship between item_weight and item_identifier  
 * item_identifier and item_weight seem to have a 1 to 1 correlation
 * fill NaN with mode of group

In [8]:
# filter dataframe
df.loc[df['Item_Identifier'] == 'DRA59',['Item_Identifier','Item_Weight']]

,Item_Identifier,Item_Weight
1876,DRA59,8.27
3927,DRA59,8.27
3962,DRA59,8.27
5609,DRA59,8.27
6057,DRA59,NaN
6179,DRA59,NaN
6338,DRA59,8.27
8371,DRA59,8.27


#### Explore relationship between outlet_identifier and outlet_size  
 Outlet_size is missing from 3 outlet_identifiers
  * OUT010, OUT017, OUT045
  * Outlet_Size varies with Outlet_Type and Outlet_Location_Type
  * did not observe any relationship useful to fill NaN in Outlet_Size column

In [9]:
# groupby outlet_identifier
df.groupby('Outlet_Identifier').count().sort_values('Outlet_Size')

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
Outlet_Identifier,,,,,,,,,,,
OUT010,555,555,555,555,555,555,555,0,555,555,555
OUT017,926,926,926,926,926,926,926,0,926,926,926
OUT045,929,929,929,929,929,929,929,0,929,929,929
OUT019,528,0,528,528,528,528,528,528,528,528,528
OUT018,928,928,928,928,928,928,928,928,928,928,928
OUT035,930,930,930,930,930,930,930,930,930,930,930
OUT046,930,930,930,930,930,930,930,930,930,930,930
OUT049,930,930,930,930,930,930,930,930,930,930,930
OUT013,932,932,932,932,932,932,932,932,932,932,932


In [10]:
df.groupby(['Outlet_Location_Type','Outlet_Type','Outlet_Identifier'])['Outlet_Size'].size()

Outlet_Location_Type  Outlet_Type        Outlet_Identifier
Tier 1                Grocery Store      OUT019               528
                      Supermarket Type1  OUT046               930
                                         OUT049               930
Tier 2                Supermarket Type1  OUT017               926
                                         OUT035               930
                                         OUT045               929
Tier 3                Grocery Store      OUT010               555
                      Supermarket Type1  OUT013               932
                      Supermarket Type2  OUT018               928
                      Supermarket Type3  OUT027               935
Name: Outlet_Size, dtype: int64

In [11]:
# groupby outlet_type
df.groupby(['Outlet_Type','Outlet_Location_Type','Outlet_Identifier','Outlet_Size']).size()

Outlet_Type        Outlet_Location_Type  Outlet_Identifier  Outlet_Size
Grocery Store      Tier 1                OUT019             Small          528
Supermarket Type1  Tier 1                OUT046             Small          930
                                         OUT049             Medium         930
                   Tier 2                OUT035             Small          930
                   Tier 3                OUT013             High           932
Supermarket Type2  Tier 3                OUT018             Medium         928
Supermarket Type3  Tier 3                OUT027             Medium         935
dtype: int64

### Fill Missing Values

#### Use fill method to replace missing data
 * Item_Weight can be filled with mode based on Item_identifier
 * Outlet_Size can be filled with mode based on Outlet_Identifier

In [13]:
df['Item_Weight'] = df.groupby('Item_Identifier')['Item_Weight'].apply(lambda x: x.fillna(x.mean()))

In [14]:
df['Item_Weight'].isnull().sum()

4

In [17]:
df[df['Item_Identifier'] == 'DRA59']

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
1876,DRA59,8.27,Regular,0.127928,Soft Drinks,184.8924,OUT046,1997,Small,Tier 1,Supermarket Type1,4442.2176
3927,DRA59,8.27,Regular,0.128127,Soft Drinks,183.6924,OUT049,1999,Medium,Tier 1,Supermarket Type1,1295.6468
3962,DRA59,8.27,Regular,0.127821,Soft Drinks,185.9924,OUT013,1987,High,Tier 3,Supermarket Type1,555.2772
5609,DRA59,8.27,Regular,0.000000,Soft Drinks,183.2924,OUT017,2007,NaN,Tier 2,Supermarket Type1,2406.2012
6057,DRA59,8.27,Regular,0.127308,Soft Drinks,186.6924,OUT027,1985,Medium,Tier 3,Supermarket Type3,7033.5112
6179,DRA59,8.27,Regular,0.223985,Soft Drinks,186.2924,OUT019,1985,Small,Tier 1,Grocery Store,555.2772
6338,DRA59,8.27,Regular,0.128449,Soft Drinks,186.5924,OUT018,2009,Medium,Tier 3,Supermarket Type2,4442.2176
8371,DRA59,8.27,Regular,0.214125,Soft Drinks,183.9924,OUT010,1998,NaN,Tier 3,Grocery Store,185.0924


### Expand Outlet_Type Column  
 * is this a necessary step...???
 * create column Outlet_Category
 * create column Outlet_Type

# Save File to CSV

In [18]:

filename = r"D:\CodingDojo\PandasForDataManipulation\food-sales-predictions-kb\sales_predictions_clean.csv"

df.to_csv(filename)